In [319]:
import pandas as pd
import nltk
from nltk import word_tokenize
import numpy as np

In [320]:
df = pd.read_csv('train.csv')
df.head()

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [321]:
df['words'] = df.text.apply(lambda text: word_tokenize(text.lower()))

W = df.explode('words')[['id', 'author', 'words']]
# W.to_csv("tokenized.csv")
W = W.rename(columns= {'words':'token', 'id': 'doc_id'})
W['topic'] = np.nan
W.head()




,doc_id,author,token,topic
0,id26305,EAP,this,NaN
0,id26305,EAP,process,NaN
0,id26305,EAP,",",NaN
0,id26305,EAP,however,NaN
0,id26305,EAP,",",NaN


In [322]:
k  = 10

In [323]:
V = pd.DataFrame({"token": W['token'].unique()})
for i in range(k):
    V[f'k{i}'] = 0
V.head()


,token,k0,k1,k2,k3,k4,k5,k6,k7,k8,k9
0,this,0,0,0,0,0,0,0,0,0,0
1,process,0,0,0,0,0,0,0,0,0,0
2,",",0,0,0,0,0,0,0,0,0,0
3,however,0,0,0,0,0,0,0,0,0,0
4,afforded,0,0,0,0,0,0,0,0,0,0


In [324]:
D = pd.DataFrame({"doc_id": df['id']})
for i in range(k):
    D[f'k{i}'] = 0
D.head()

,doc_id,k0,k1,k2,k3,k4,k5,k6,k7,k8,k9
0,id26305,0,0,0,0,0,0,0,0,0,0
1,id17569,0,0,0,0,0,0,0,0,0,0
2,id11008,0,0,0,0,0,0,0,0,0,0
3,id27763,0,0,0,0,0,0,0,0,0,0
4,id12958,0,0,0,0,0,0,0,0,0,0


In [333]:
W['topic'] = np.random.choice(np.arange(10), len(W),  p=[1/k]*k)
new_df = pd.DataFrame(W.groupby(by=['doc_id', 'topic'], as_index=False)['token'].count()).rename(columns = {"topic": "topic_index", "token": "count"})
new_df['topic'] = new_df['topic_index'].apply(lambda topic: f'k{topic}')
D = new_df.pivot(index='doc_id', columns='topic', values='count').fillna(0)
Ds = D.sum(axis='columns')
Ds.transpose()

doc_id
id00001    10.0
id00002    18.0
id00003    20.0
id00004    31.0
id00005    15.0
           ... 
id27965    31.0
id27966    13.0
id27967    55.0
id27968    57.0
id27971    27.0
Length: 19579, dtype: float64

In [326]:
new_df = pd.DataFrame(W.groupby(by=['token', 'topic'], as_index=False)['doc_id'].count()).rename(columns={"topic": "topic_index", "doc_id": "count"})
new_df['topic'] = new_df['topic_index'].apply(lambda topic: f'k{topic}')
new_df = new_df.pivot(index='token', columns='topic', values='count').fillna(0)
V= new_df
Vs = V.sum(axis="columns")

In [331]:
# start main sampling loop
alpha =  min(0.1,50/k)
beta = 0.01

for it in range(5):
    for i in range(len(W)):
        if i % 1001 == 0:
            print('iteration:', it, 'in, token:', i, 'of', len(W))
        row = W.iloc[i]
        V.loc[row['token'], f'k{int(row["topic"])}'] -= 1
        D.loc[row['doc_id'], f'k{int(row["topic"])}'] -= 1
        Vs[row["token"]] -= 1
        Ds[row["doc_id"]] -= 1
        p = np.array([(alpha + D.loc[row['doc_id'], f'k{j}'])/(k*alpha + Ds[row['doc_id']])*(beta + V.loc[row['token'], f'k{j}'])/(len(V)*beta + Vs[row['token']]) for j in range(k)])
        p = p/p.sum()
        new_topic = int(np.random.choice(np.arange(10),  p=[1/k]*k))
        W.at[i, 'topic'] = new_topic
        V.loc[row['token'], f'k{new_topic}'] += 1
        D.loc[row['doc_id'], f'k{new_topic}'] += 1
        Vs[row["token"]] += 1
        Ds[row["doc_id"]] += 1
    print('iteration comlete', it)
        

iteration: 0 in, token: 0 of 596971
iteration: 0 in, token: 1001 of 596971
iteration: 0 in, token: 2002 of 596971
iteration: 0 in, token: 3003 of 596971
iteration: 0 in, token: 4004 of 596971
iteration: 0 in, token: 5005 of 596971
iteration: 0 in, token: 6006 of 596971
iteration: 0 in, token: 7007 of 596971
iteration: 0 in, token: 8008 of 596971
iteration: 0 in, token: 9009 of 596971
iteration: 0 in, token: 10010 of 596971
iteration: 0 in, token: 11011 of 596971
iteration: 0 in, token: 12012 of 596971
iteration: 0 in, token: 13013 of 596971
iteration: 0 in, token: 14014 of 596971
iteration: 0 in, token: 15015 of 596971
iteration: 0 in, token: 16016 of 596971
iteration: 0 in, token: 17017 of 596971
iteration: 0 in, token: 18018 of 596971
iteration: 0 in, token: 19019 of 596971
iteration: 0 in, token: 20020 of 596971
iteration: 0 in, token: 21021 of 596971
iteration: 0 in, token: 22022 of 597365
iteration: 0 in, token: 23023 of 598366
iteration: 0 in, token: 24024 of 599367
iteration: 0 

KeyboardInterrupt: 

In [ ]:
W.head()